## Unix Pipes
### Bioinformatics Coffee Hour - June 16, 2020
#### Your Host: Nathan Weeks

## What this lesson will cover
* Unix pipes in shell scripts to improve efficiency of simple bioinformatics workflows
* _Example:_ Short-read (FASTQ) alignment to BAM 

### Software Utilities Used

#### curl
    https://curl.haxx.se/
command line tool and library for transferring data with URLs

#### gzip
    https://www.gnu.org/software/gzip/
popular data compression program

#### fastp
    https://github.com/OpenGene/fastp
all-in-one FASTQ preprocessor (QC/adapters/trimming/filtering/splitting/merging...)

#### bwa
    http://bio-bwa.sourceforge.net/
software package for mapping low-divergent sequences against a large reference genome

#### samtools
    https://github.com/samtools/samtools
tools... for manipulating [Reading/writing/editing/indexing/viewing] next-generation sequencing data [SAM/BAM/CRAM format]

## Motivation
Bioinformatics workflows are usually *big*...
  - big input files
  - many workflow **stages**
    + each workflow stage usually takes as input a file(s) from the previous stage, and produces output file(s) for the next stage
    + each stage may use a different software tool

## Example: 1/2 of a variant-calling pipeline
<img src='https://github.com/harvardinformatics/shortRead_mapping_variantCalling/raw/3965d337e41b6d5fc8b873d7d485fb4bf8528bee/docs/workflowSchematic_fastq2bam.png'>

*source: Brian Arnold, https://github.com/harvardinformatics/shortRead_mapping_variantCalling/ *

### Definition
<dl>
    <dt>file</dt>
    <dd>An object that can be written to, or read from, or both.</dd>
</dl>

*source: POSIX.1-2018*

#### Unix processes start with 3 open "files" (connected to a terminal in an interactive shell)
<dl>
    <dt>standard input ("stdin")</dt>
    <dd>An input stream usually intended to be used for primary data input.</dd>
    <dt>standard output ("stdout")<dt>
    <dd>An output stream usually intended to be used for primary data output.</dd>
    <dt>standard error ("stderr")<dt>
    <dd>An output stream usually intended to be used for diagnostic messages.<dd>
</dl>

![process](images/process.svg)

<dl>
    <dt>filter</dt>
    <dd>A command whose operation consists of reading data from standard input or a list of input files and writing data to standard output.
        Typically, its function is to perform some transformation on the data stream.</dd>
</dl>

source: POSIX.1-2018_

_Many unix programs act as filters_

## Example: grep

When given only a pattern argument (no file arguments), grep operates as a _filter_, reading input from stdin, and writing output to stdout (_enter CTRL-D to indicate end of input_)

```
$ grep PATTERN
```

### Redirect stdout to a file

To write grep's output to a file (instead of a terminal), use the shell redirection operator (`>`)

```
$ grep PATTERN > matched.txt
```

### Introducing Pipes
Pipes are a form of [Interprocess Communication](https://en.wikipedia.org/wiki/Inter-process_communication) with the following properties:
* One-way commmunication channel between two _processes_
  - **process**: a running program
* Data written (sequentially) by one process (usually to its stdout) is read (sequentially) by another (usually from its stdin)
* Operating system kernel blocks writer (process in a "sleep" state) until data is read by reader
  + Similarly, reader blocks until writer writes data

## Why use pipes?
### Example: Sequential Laundry
![laundry1](images/laundry1.gif)
*source: https://cs.stanford.edu/people/eroberts/courses/soco/projects/risc/pipelining/index.html*

## Why use pipes?
### Example: Pipelined Laundry
![laundry2](images/laundry2.gif)
*source: https://cs.stanford.edu/people/eroberts/courses/soco/projects/risc/pipelining/index.html*

## Pipes in Programming Languages 
* The (Unix/Linux/POSIX) shell command language (understood by bash, zsh, sh, and others) has a construct called a pipe (`|` operator)
* Many other programming languages allow use of pipes in some fashion:
  - AWK: `command | getline [var]` to read, `print ... > command` to 
  - Python: [subprocess.Popen()](https://docs.python.org/3/library/subprocess.html#replacing-shell-pipeline)
  - R: `pipe()`
    + Not to be confused with the pipe `%>%` operator from the magrittr package

### (shell) Pipeline
* A sequence of commands separated by the `|` (pipe) operator
* standard output of the command on the left of the `|` is connected to the standard input of the command to the right of the `|`

---
## Examples
---

#### Example 1: download FASTA file and compress on-the-fly

In [ ]:
mkdir -p input
GENOME_FASTA_URL=https://github.com/harvardinformatics/shortRead_mapping_variantCalling/raw/b120a823c23b1eaf1cfb95ea5d5ca0ce26a50c32/data/genome/Tgut_subseg_renamed.fa
curl -sL ${GENOME_FASTA_URL} | gzip -v -9 > input/Tgut_subseg_renamed.fa.gz
ls -lh input/Tgut_subseg_renamed.fa.gz

##### Explanation: 
* `curl` downloads the FASTA file from the specified URL and writes it to its stdout.
* `gzip`, when not given a file argument, reads (uncompressed) data from stdin and writes compressed data to stdout.
  - The `gzip -v` option prints the compression ratio to stderr_, the `-9` option specififies maximal compression (==more run time, but better compression)
* The shell output redirection operator (`>`) writes the stdout of `gzip` to a file (Tgut_subseg_renamed.fa.gz)

#### Example 2: Print the length of each sequence in the compressed FASTA file
Our FASTA file has one sequence per line. [AWK](https://en.wikipedia.org/wiki/AWK) can generate a tab-separated list of sequence ID and sequence length (

_Problem:_ AWK cannot read a gzip file directly.

_Solution:_ Combine `gzip` options`-d` (decompress) and `-c` (write to stdout), and pipe to awk's stdin:

In [ ]:
gzip -dc input/Tgut_subseg_renamed.fa.gz |
  awk '/^>/ { if (len) print len; len = 0; printf("%s\t", substr($1,2)) }
       !/^>/ { len+=length }
       END { printf("%i\n", len) }'

Fetch sequence reads to align:

In [ ]:
(cd input && curl -LO 'https://github.com/harvardinformatics/shortRead_mapping_variantCalling/raw/5530f1991da66af82d0213bc3492da8a431a1a92/data/fastq/ERR1013163_[1-2].fastq.gz')

Next, index the genome file so `bwa` can align reads to it:

_Note: `bwa` can index (gzip-)compressed reference sequences_

In [ ]:
bwa index input/Tgut_subseg_renamed.fa.gz

### Sequence alignment workflow (sequential)

* Output of each stage written as file(s)
* Command in each stage must be run to completion before next stage is executed

In [ ]:
set -o xtrace
mkdir -p orig
time (
cd orig
ln -sf ../input
# trim reads, generate HTML report
fastp --in1 input/ERR1013163_1.fastq.gz --in2 input/ERR1013163_2.fastq.gz --out1 ERR1013163_1.fastq-trimmed.gz --out2 ERR1013163_2.fastq-trimmed.gz
# align reads
bwa mem input/Tgut_subseg_renamed.fa.gz ERR1013163_1.fastq-trimmed.gz ERR1013163_2.fastq-trimmed.gz > ERR1013163.sam
# sort alignments by read name
samtools sort -n --output-fmt bam -o ERR1013163.bam ERR1013163.sam 
# fill in mate coordinates and insert size in BAM records
samtools fixmate -m ERR1013163.bam ERR1013163-fixmate.bam
# sort by alignment coordinate
samtools sort -o ERR1013163-fixmate-sort.bam ERR1013163-fixmate.bam
# mark duplicate alignments
samtools markdup -f markdup.stats --output-fmt-option level=9 ERR1013163-fixmate-sort.bam out.bam
# index resulting bam file 
samtools index out.bam
samtools stats out.bam > out.bam.stats.txt
)


Check files created :

In [ ]:
ls -lh orig

## A special "file": /dev/stdin

* Some utilities either accept an optional file argument (e.g., gzip, awk, most other unix utilities), reading from stdin if no input file is specified
* Some utilities accept "-" in place of either an input or output file argument to mean stdin (or stdout) respectively
* Other utilities require an actual file argument; e.g. `bwa mem`:
```
      bwa mem [options] db.prefix reads.fq [mates.fq]
```
  - **reads.fq** is required. However, `bwa mem` reads **reads.fq** sequentially, and can conceptually use a pipe.
  
```
    fastp reads.fq | bwa mem db.prefix /dev/stdin
```
  
**Note: this works only if the utility reads (or writes) the file sequentially**

### Sequence alignment workflow (pipeline)

* Process in each stage writes output directly to input of another process(es)
* Processes execute concurrently

In [ ]:
mkdir -p ex-pipe
time (
cd ex-pipe
ln -s ../input
# fastp supports interleaved (https://github.com/OpenGene/fastp#output-to-stdout) output
fastp --in1 input/ERR1013163_1.fastq.gz --in2 input/ERR1013163_2.fastq.gz --stdout 2>/dev/null |
  # bwa mem requires a FASTQ file operand <in1.fq> (will not take reads from stdin).
  # /dev/stdin is a special file that represents a process's standard input
  bwa mem input/Tgut_subseg_renamed.fa.gz /dev/stdin 2> /dev/null |
    # sort input SAM records by read name ("-n"), write uncompressed BAM ((compression) level=0) to stdout
    # (otherwise, samtools will detect input format & use same output format)
    samtools sort -n --output-fmt bam --output-fmt-option level=0 |
      # Usage: samtools fixmate <in.nameSrt.bam> <out.nameSrt.bam>
      # samtools allows "-" to be used in place of 
      samtools fixmate -m - - |
        # samtools sort reads from stdin and writes to stdout by default
        samtools sort |
          samtools markdup -f markdup.stats --output-fmt-option level=9 - - |
            # samtools index requires an input bam file argument; use /dev/stdin
            tee out.bam >(samtools index /dev/stdin out.bam.bai) |
              # samtools stats reads from stdin and writes to stdout by default
              samtools stats > out.bam.stats.txt
)

Much less data written to / read from disk:

In [ ]:
ls -lh ex-pipe

Let's take a closer look at the last stages of the pipeline.

Two additional tools are being introduced: the [tee](https://en.wikipedia.org/wiki/Tee_(command)) command, and [process substitution](https://en.wikipedia.org/wiki/Process_substitution) ( `>(...)` )

### tee

`tee` duplicates its stdin to one or more files in addition to stdout:

![tee](https://upload.wikimedia.org/wikipedia/commons/2/24/Tee.svg)

*source: Wikipedia*

```
... |
  samtools markdup -f markdup.stats --output-fmt-option level=9 - - |
    tee out.bam >(samtools index /dev/stdin out.bam.bai) |
      samtools stats > out.bam.stats.txt
```
![pipe](images/pipe.svg)

# The End!
## Questions, comments...?

---
## Bonus material
---

### pipefail
By convention, processes return an exit status of 0 to indicate success, and > 0 to indicate failure.

By default, the exit status of a pipeline is the exit status of the last command in the pipeline:

In [ ]:
samtools sort foo/bar.sam | head
echo "exit status: $?"

This is inconvenient when trying to determine if a (Slurm) job script ran successfully, or if it's desired to terminate the script if any command fails (e.g., using `set -o errexit`).

The `pipefail` option causes the exit status of a pipeline to be the exit status of the last command in the pipeline to have a non-zero exit status:

In [ ]:
set -o pipefail
samtools sort foo/bar.sam | head
echo "exit status: $?"

## Other Terms

### Regular File
> A file that is a _**randomly accessible**_ sequence of bytes.
```
$ ls -lh ERR1013163_1.fastq.gz
-rw-r--r-- 1 jovyan root 3.8M Jun 10 16:00 ERR1013163_1.fastq.gz

### FIFO (aka "named pipe")
**F**irst **I**n **F**irst **O**ut

> A type of file with the property that data written to such a file is read on a first-in-first-out basis.

_In other words, data are written / read **sequentially**_

### FIFO Example

In [ ]:
set -o xtrace
mkfifo test.fifo
printf 'line1\nline2\nline3\n' > test.fifo &
sleep 5
jobs
sleep 5
cat test.fifo

### Process Substitution
`command1 >(command2)` is loosely analogous to:
```
mkfifo tmp.fifo
command1 > tmp.fifo &
command2 < tmp.fifo
rm tmp.fifo
```